#### General Steps to Follow

1. Importing Packages
2. Loading the model
3. Collecting verification images
4. Preprocessing the input image
5. Defining a verification function
6. Getting the input and verify

## 1) Importing Packages

In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Layer

### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 2) Loading the model

#### The distance layer

In [2]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super(L1Dist, self).__init__(**kwargs)

    def call(self, anchor, validation):
        return tf.abs(anchor - validation)

In [3]:
siamese_model = tf.keras.models.load_model('../my data/siamese_model.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 3) Collecting verification images

#### Setup Paths

In [4]:
inp_img_path = "../my data/input_image" # Path for the input image that will be used in verification
ver_path = "../my data/verification"   # Path for some anchor images will be used to verify the input images

#### Collecting the verification images

In [5]:
ver_images = []
for img_name in os.listdir(ver_path):
    path = ver_path + "/" + img_name
    image = plt.imread(path)
    ver_images.append(image)
ver_images = np.array(ver_images)

print("Shape of verification images is:", ver_images.shape)

Shape of verification images is: (73, 250, 250, 3)


#### Normalizing the verification images

In [6]:
ver_images = ver_images/ [255]

#### Resizing the verification images to (105,105,3)

In [7]:
temp = []
for i in range(ver_images.shape[0]):
    img = ver_images[i]
    img = tf.image.resize(img,(105,105))
    img = img.numpy()
    temp.append(img)
temp = np.array(temp)
temp = temp.reshape((73,1,105,105,3))
ver_images = temp

print("Shape of the resized verification images is:", ver_images.shape)

Shape of the resized verification images is: (73, 1, 105, 105, 3)


### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 4) Preprocessing the input image

In [8]:
def image_transformation(img):
    # Normalizing the image
    img = img/255
    # resizing the image to (105, 105, 3)
    img = tf.image.resize(img,(105,105))
    img = img.numpy()
    img = img.reshape((1,105,105,3))
    
    return img

### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 5) Defining a verification function

In [9]:
def verify(input_image):
    print("\nNumber of images will be used in verification is:", ver_images.shape[0])
    print("\n")
    
    result = []
    i = 1
    for ver in ver_images:
        print("Verification Image #", i); i+=1
        y_hat = siamese_model.predict([ver,input_image])
        if(y_hat[0,0] >= 0.5):
            y_hat[0,0] = 1
        else:
            y_hat[0,0] = 0
        result.append(y_hat[0,0])
    
    m = ver_images.shape[0]
    result = np.array(result)
    accuracy = 100*(np.sum(result == 1)/m)
    
    print("\nOutput:")
    print("Accuracy =", accuracy)
    if(accuracy > 60):
        print("Verified, Welcome Amro!")
    else:
        print("Not Recognized")

### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 6) Getting the input and verify

In [13]:
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px
    frame = frame[120:120+250,200:200+250, :]
    input_image = frame.copy()
    
    # Show image back to screen
    cv2.imshow('Verification Image', frame)
    
    # Normalizing and resizing the image
    input_image = image_transformation(input_image)

    
    if cv2.waitKey(1) & 0XFF == ord('a'): 
        imgname = inp_img_path + "/" + "input_image.jpg"

        # Write out input image
        cv2.imwrite(imgname, frame)
                
        # Verification
        verify(input_image)
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()


Number of images will be used in verification is: 73


Verification Image # 1
1/1 [==============================] - 0s 460ms/step
Verification Image # 2
1/1 [==============================] - 0s 242ms/step
Verification Image # 3
1/1 [==============================] - 0s 237ms/step
Verification Image # 4
1/1 [==============================] - 0s 240ms/step
Verification Image # 5
1/1 [==============================] - 0s 222ms/step
Verification Image # 6
1/1 [==============================] - 0s 227ms/step
Verification Image # 7
1/1 [==============================] - 0s 243ms/step
Verification Image # 8
1/1 [==============================] - 0s 242ms/step
Verification Image # 9
1/1 [==============================] - 0s 240ms/step
Verification Image # 10
1/1 [==============================] - 0s 239ms/step
Verification Image # 11
1/1 [==============================] - 0s 249ms/step
Verification Image # 12
1/1 [==============================] - 0s 239ms/step
Verification Image # 13
1/1 [

1/1 [==============================] - 0s 246ms/step
Verification Image # 33
1/1 [==============================] - 0s 251ms/step
Verification Image # 34
1/1 [==============================] - 0s 246ms/step
Verification Image # 35
1/1 [==============================] - 0s 252ms/step
Verification Image # 36
1/1 [==============================] - 0s 245ms/step
Verification Image # 37
1/1 [==============================] - 0s 244ms/step
Verification Image # 38
1/1 [==============================] - 0s 244ms/step
Verification Image # 39
1/1 [==============================] - 0s 223ms/step
Verification Image # 40
1/1 [==============================] - 0s 242ms/step
Verification Image # 41
1/1 [==============================] - 0s 240ms/step
Verification Image # 42
1/1 [==============================] - 0s 243ms/step
Verification Image # 43
1/1 [==============================] - 0s 229ms/step
Verification Image # 44
1/1 [==============================] - 0s 244ms/step
Verification Image # 45

1/1 [==============================] - 0s 223ms/step
Verification Image # 65
1/1 [==============================] - 0s 224ms/step
Verification Image # 66
1/1 [==============================] - 0s 243ms/step
Verification Image # 67
1/1 [==============================] - 0s 242ms/step
Verification Image # 68
1/1 [==============================] - 0s 238ms/step
Verification Image # 69
1/1 [==============================] - 0s 243ms/step
Verification Image # 70
1/1 [==============================] - 0s 241ms/step
Verification Image # 71
1/1 [==============================] - 0s 238ms/step
Verification Image # 72
1/1 [==============================] - 0s 241ms/step
Verification Image # 73
1/1 [==============================] - 0s 240ms/step

Output:
Accuracy = 93.15068493150685
Verified, Welcome Amro!
